In [13]:
!pip install opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl.metadata (60 kB)
Using cached opencv_python-4.12.0.88-cp37-abi3-win_amd64.whl (39.0 MB)
Using cached numpy-2.2.6-cp312-cp312-win_amd64.whl (12.6 MB)

  Attempting uninstall: numpy

    Found existing installation: numpy 1.26.4

   ---------------------------------------- 0/2 [numpy]
    Uninstalling numpy-1.26.4:
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 

  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.
lmdeploy 0.9.1 requires numpy<2.0.0, but you have numpy 2.2.6 which is incompatible.


In [ ]:
import cv2
from pathlib import Path
from typing import List

def capture_frames_near_timestamps(
    video_path: str,
    timestamps_sec: List[float],
    output_dir: str = "frames_out"
):
    
    video_path = Path(video_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise RuntimeError(f"비디오를 열 수 없습니다: {video_path}")

    # 영상 정보 확인
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if fps <= 0:
        raise RuntimeError(f"FPS를 가져올 수 없습니다. fps={fps}")

    duration = frame_count / fps
    print(f"[INFO] 파일: {video_path}")
    print(f"[INFO] FPS = {fps}, 총 프레임 수 = {frame_count}, 길이 ≈ {duration:.3f}초")

    base_name = video_path.stem

    for idx, ts in enumerate(timestamps_sec):
        # 타임스탬프가 영상 범위 밖이면 스킵
        if ts < 0 or ts > duration:
            print(f"[WARN] {ts:.3f}초는 영상 길이(0~{duration:.3f}) 밖입니다. 스킵.")
            continue

        # 타임스탬프 → 프레임 인덱스 (근처 프레임이면 OK니까 반올림)
        frame_idx = int(round(ts * fps))
        # 프레임 수를 넘지 않도록 클램프
        frame_idx = max(0, min(frame_idx, frame_count - 1))

        # 해당 프레임으로 위치 이동
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        print(f"[DEBUG] ts={ts:.3f}s, frame_idx={frame_idx}, ret={ret}")

        if not ret or frame is None:
            print(f"[WARN] {ts:.3f}s 근처 프레임을 읽지 못했습니다.")
            continue

        # 저장 파일 이름
        safe_ts = str(round(ts, 3)).replace('.', '_')
        filename = f"{base_name}_{idx:03d}_{safe_ts}s.jpg"
        out_path = output_dir / filename

        saved = cv2.imwrite(str(out_path), frame)
        if saved:
            print(f"{ts:.3f}s 근처 프레임 저장: {out_path}")
        else:
            print(f"{out_path} 저장 실패!")

    cap.release()
    print("프레임 캡처 완료.")



video_file = "hanhwa_samsung_PO.mp4" # 동영상 파일 경로
# 초 단위 타임스탬프
timestamps_sec = [50.689, 62.96, 81.379, 104.004, 114.852, 140.175, 227.655, 311.635, 331.863, 403.47, 436.64, 490.56, 513.06, 556.64, 604.7, 661.27, 690.015, 739.54, 752.66]
output_dir = "frames" # 이미지 저장 폴더

capture_frames_near_timestamps(video_file, timestamps_sec, output_dir)


[INFO] 파일: hanhwa_samsung_PO.mp4
[INFO] FPS = 59.94005994005994, 총 프레임 수 = 74370, 길이 ≈ 1240.740초
[DEBUG] ts=50.689s, frame_idx=3038, ret=True
[INFO] 50.689s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_000_50_689s.jpg
[DEBUG] ts=62.960s, frame_idx=3774, ret=True
[INFO] 62.960s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_001_62_96s.jpg
[DEBUG] ts=81.379s, frame_idx=4878, ret=True
[INFO] 81.379s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_002_81_379s.jpg
[DEBUG] ts=104.004s, frame_idx=6234, ret=True
[INFO] 104.004s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_003_104_004s.jpg
[DEBUG] ts=114.852s, frame_idx=6884, ret=True
[INFO] 114.852s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_004_114_852s.jpg
[DEBUG] ts=140.175s, frame_idx=8402, ret=True
[INFO] 140.175s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_005_140_175s.jpg
[DEBUG] ts=227.655s, frame_idx=13646, ret=True
[INFO] 227.655s 근처 프레임 저장: frames_example\hanhwa_samsung_PO_006_227_655s.jpg
[DEBUG] ts=311.635s, frame_idx=18679, ret=True
[INF